# 1. 导入环境

本节将导入训练DeepSeek-7B-chat模型所需的所有Python库和依赖包，并进行基础的环境配置。

### 主要依赖库说明

#### 核心机器学习库
- **transformers**: Hugging Face的Transformers库，用于加载预训练模型和分词器
- **torch**: PyTorch深度学习框架，提供模型训练的基础功能
- **datasets**: Hugging Face的数据集处理库，用于高效处理训练数据
- **peft**: Parameter-Efficient Fine-Tuning库，提供LoRA等高效微调方法

#### 数据处理库
- **pandas**: 数据分析和处理库，用于读取JSON数据并转换格式
- **numpy**: 数值计算库（通过其他库间接使用）

#### 训练相关组件
- **AutoTokenizer**: 自动选择合适的分词器
- **AutoModelForCausalLM**: 自动加载因果语言模型
- **DataCollatorForSeq2Seq**: 序列到序列任务的数据整理器
- **TrainingArguments**: 训练参数配置类
- **Trainer**: Hugging Face的训练器，简化训练流程
- **GenerationConfig**: 文本生成配置类

### 环境要求
- Python 3.8+
- CUDA 11.8+ (用于GPU训练)
- 至少24GB GPU内存 (推荐24GB+)
- 足够的磁盘空间存储模型和数据集
- GPU驱动和CUDA工具包已正确安装

In [ ]:
# 导入相应的依赖
from datasets import Dataset
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer, GenerationConfig

# 2.读取和处理数据集
### 2.1 读取数据

In [87]:
# 读取对应的数据
file_path = './dataset/tianji-chinese/tianji-wishes-chinese-v0.1-format.json'
df = pd.read_json(file_path)
ds = Dataset.from_pandas(df)
ds[:3]

{'instruction': ['你现在是一个送祝福大师，帮我针对不同人和事情、节日送对应的祝福',
  '你现在是一个送祝福大师，帮我针对不同人和事情、节日送对应的祝福',
  '你现在是一个送祝福大师，帮我针对不同人和事情、节日送对应的祝福'],
 'input': ['我想送赵老师生日祝福,严肃风格', '送祝福给赵老师春节,严肃风格', '祝赵老师元宵节快乐,严肃风格'],
 'output': ['尊敬的赵老师，值此生辰之际，愿岁月如诗，为您带来无尽的喜悦与美好；愿时光荏苒，为您留下珍贵的回忆与感悟。愿您快快乐乐，事业更上一层楼，教诲之恩桃李满天下。在这充满敬意的时刻，恭祝赵老师生日快乐，幸福安康！',
  '赵老师，值此春节佳节之际，恭祝您福寿安康，万事如意。在过去的一年里，您的辛勤耕耘为后辈树立了榜样，新春到来，愿您的生活如诗如画，工作更上一层楼，继续以您的智慧和热忱，引领我们前行。岁月静好，愿您享受每一个温馨时刻，幸福安康，喜悦无忧。',
  '尊敬的赵老师，元宵佳节至，愿您福寿安康，智慧如灯。愿这团圆的灯火，照亮您的事业与前程，带来更多学术的辉煌与成就。在这温馨的时刻，愿您家庭美满，幸福长存，如元宵般圆满甜蜜。谨祝元宵快乐，万事如意！']}

### 2.2 加载分词器

In [88]:
# 初始化模型的分词器
tokenizer = AutoTokenizer.from_pretrained('./deepseek-ai/deepseek-llm-7b-chat/', use_fast=False, trust_remote_code=True)
# 设置填充方向为右侧填充，不会影响模型对序列开头的理解
tokenizer.padding_side = 'right'
tokenizer

LlamaTokenizerFast(name_or_path='./deepseek-ai/deepseek-llm-7b-chat/', vocab_size=100000, model_max_length=4096, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<｜begin▁of▁sentence｜>', 'eos_token': '<｜end▁of▁sentence｜>', 'pad_token': '<｜end▁of▁sentence｜>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	100000: AddedToken("<｜begin▁of▁sentence｜>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	100001: AddedToken("<｜end▁of▁sentence｜>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	100002: AddedToken("ø", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	100003: AddedToken("ö", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	100004: AddedToken("ú", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	100005: AddedToken("ÿ", rstrip=False, lstrip=False, single_word=False, normali

### 2.3 数据格式化
Lora 训练的数据是需要经过格式化、编码之后再输入给模型进行训练的，将数据编码成多维向量。

In [97]:
def process_func(example):
    MAX_LENGTH = 384    # Llama分词器会将一个中文字切分为多个token，因此需要放开一些最大长度，保证数据的完整性
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer(f"User: {example['instruction']+example['input']}\n\n", add_special_tokens=False)  # add_special_tokens 不在开头加 special_tokens
    response = tokenizer(f"Assistant: {example['output']}<｜end▁of▁sentence｜>", add_special_tokens=False)
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
    # print("instruction", instruction)
    # print("response", response)
    # print("input_ids", input_ids)
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]  # 因为eos token咱们也是要关注的所以 补充为1
    # print("attention_mask", attention_mask)
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.pad_token_id]  
    if len(input_ids) > MAX_LENGTH:  # 做一个截断
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }
tokenized_id = ds.map(process_func, remove_columns=ds.column_names)
tokenized_id

Map: 100%|██████████| 2976/2976 [00:01<00:00, 2236.01 examples/s]


Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 2976
})

In [96]:
input_example = tokenizer.decode(tokenized_id[0]['input_ids'])
output_example = tokenizer.decode(list(filter(lambda x: x != -100, tokenized_id[1]["labels"])))
print("Input Example:", input_example)
print("Output Example:", output_example)

Input Example: User: 你现在是一个送祝福大师，帮我针对不同人和事情、节日送对应的祝福我想送赵老师生日祝福,严肃风格

Assistant: 尊敬的赵老师，值此生辰之际，愿岁月如诗，为您带来无尽的喜悦与美好；愿时光荏苒，为您留下珍贵的回忆与感悟。愿您快快乐乐，事业更上一层楼，教诲之恩桃李满天下。在这充满敬意的时刻，恭祝赵老师生日快乐，幸福安康！<｜end▁of▁sentence｜><｜end▁of▁sentence｜>
Output Example: Assistant: 赵老师，值此春节佳节之际，恭祝您福寿安康，万事如意。在过去的一年里，您的辛勤耕耘为后辈树立了榜样，新春到来，愿您的生活如诗如画，工作更上一层楼，继续以您的智慧和热忱，引领我们前行。岁月静好，愿您享受每一个温馨时刻，幸福安康，喜悦无忧。<｜end▁of▁sentence｜><｜end▁of▁sentence｜>


# 4.创建模型
模型以半精度形式加载，如果你的显卡比较新的话，可以用torch.bfolat形式加载。对于自定义的模型一定要指定trust_remote_code参数为True

In [98]:
import torch
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "7" 

model = AutoModelForCausalLM.from_pretrained('./deepseek-ai/deepseek-llm-7b-chat/', trust_remote_code=True, torch_dtype=torch.half, device_map="auto")
model.generation_config = GenerationConfig.from_pretrained('./deepseek-ai/deepseek-llm-7b-chat/')
model.generation_config.pad_token_id = model.generation_config.eos_token_id
model

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.48s/it]


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(102400, 4096)
    (layers): ModuleList(
      (0-29): 30 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-06)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-06)
      )
    )
    (norm): LlamaRMSNorm((4096,), eps=1e-06)
    (rotary_

In [101]:
# 开启梯度检查点时，要执行该方法
model.enable_input_require_grads()
# 查看模型的dtype
model.dtype

torch.float16

# 5. 定义LoraConfig
LoraConfig这个类中可以设置很多参数，但主要的参数没多少，简单讲一讲，感兴趣的同学可以直接看源码。

- `task_type`：模型类型
- `target_modules`：需要训练的模型层的名字，主要就是`attention`部分的层，不同的模型对应的层的名字不同，可以传入数组，也可以字符串，也可以正则表达式。
- `r`：`lora`的秩，控制低秩分解的维度
- `lora_alpha`：`Lora alaph`，控制LoRA权重的缩放强度
- `lora_dropout`：`Lora`的dropout，防止过拟合
- `inference_mode`：推理模式，控制是否为推理

In [14]:
from peft import LoraConfig, TaskType, get_peft_model

config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, 
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    inference_mode=False, # 训练模式
    r=8, # Lora 秩
    lora_alpha=32, # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1# Dropout 比例
)
config

LoraConfig(task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, inference_mode=False, r=8, target_modules={'k_proj', 'up_proj', 'o_proj', 'gate_proj', 'down_proj', 'v_proj', 'q_proj'}, exclude_modules=None, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', trainable_token_indices=None, loftq_config={}, eva_config=None, corda_config=None, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False)

In [ ]:
model = get_peft_model(model, config)
print(model)
parameters = model.print_trainable_parameters()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(102400, 4096)
        (layers): ModuleList(
          (0-29): 30 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear(
 

# 6. 配置训练参数
TrainingArguments是Hugging Face Transformers库中用于配置模型训练过程的核心类，它包含了训练过程中的各种超参数和设置。

### 核心参数解释
- `output_dir`: 模型和训练结果保存的目录
- `learning_rate`: 学习率，控制模型参数更新的步长
- `per_device_train_batch_size`: 每个设备的训练批次大小
- `num_train_epochs`: 训练的总轮数
- `weight_decay`: 权重衰减，用于防止过拟合
- `logging_steps`: 日志记录的步数间隔
- `save_steps`: 模型保存的步数间隔
- `gradient_checkpointing`: 是否启用梯度检查点，减少内存使用

In [104]:
args = TrainingArguments(
    output_dir="./output/DeepSeek",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    logging_steps=10,
    num_train_epochs=3,
    save_steps=100,
    learning_rate=1e-4,
    save_on_each_node=True,
    gradient_checkpointing=True
)

In [33]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_id,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [34]:
trainer.train()

Step,Training Loss
10,1.815500
20,1.469200
30,1.413400
40,1.366100
50,1.330500
60,1.301400
70,1.300900
80,1.242000
90,1.274400
100,1.242800


TrainOutput(global_step=558, training_loss=1.058647850508331, metrics={'train_runtime': 522.8689, 'train_samples_per_second': 17.075, 'train_steps_per_second': 1.067, 'total_flos': 4.116561919082496e+16, 'train_loss': 1.058647850508331, 'epoch': 3.0})

# 7. 模型评估

In [105]:
# text = "小姐，别的秀女都在求中选，唯有咱们小姐想被撂牌子，菩萨一定记得真真儿的——"
text = "祝姐姐生日快乐, 小红书风格"
inputs = tokenizer(f"User: {text}\n\n", return_tensors="pt")
outputs = model.generate(**inputs.to(model.device), max_new_tokens=100)

result = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(result)


User: 祝姐姐生日快乐, 小红书风格

尊敬的姐姐，今天是你的生日，我祝你生日快乐！在这个特别的日子里，我想对你说，你是我生命中最重要的人之一，你的存在让我的生活变得更加美好。我感谢你一直以来对我的关心和帮助，你的支持和鼓励是我前进的动力。

祝你生日快乐！愿你的生活充满阳光、鲜花和快乐，愿你拥有健康、幸福和成功。希望你在未来的日子里，能够实现自己的梦想，追求自己的幸福。

最后


In [106]:
deepseek_model = AutoModelForCausalLM.from_pretrained('./deepseek-ai/deepseek-llm-7b-chat/', trust_remote_code=True, torch_dtype=torch.half, device_map="auto")
deepseek_model.generation_config = GenerationConfig.from_pretrained('./deepseek-ai/deepseek-llm-7b-chat/')
deepseek_model.generation_config.pad_token_id = deepseek_model.generation_config.eos_token_id

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.39s/it]


In [107]:
text = "祝姐姐生日快乐, 小红书风格"
inputs = tokenizer(f"User: {text}\n\n", return_tensors="pt")
outputs = deepseek_model.generate(**inputs.to(deepseek_model.device), max_new_tokens=512)
result = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(result)

User: 祝姐姐生日快乐, 小红书风格

🎉🎂🎁🎁🎁🎂🎉

亲爱的小红书宝宝们，今天我要和大家分享一件特别开心的事情，我的姐姐今天过生日啦！🎉🎂🎈

姐姐是我生命中最重要的人之一，她不仅是我最亲的亲人，也是我最可靠的朋友。记得小时候，每次我遇到困难和挫折，姐姐总是第一个站出来支持我，鼓励我，帮助我。她的笑容，她的温暖，一直是我生命中最美好的回忆。🥰

所以今天，我想通过小红书这个平台，向全世界大声地说：姐姐，生日快乐！希望你的每一天都能像今天一样，充满欢笑和幸福。🎉🎂🎈

当然，作为你的亲妹妹，我也要送上一份特别的生日礼物，希望你能够喜欢。不过，在这里我就不剧透啦，让我们一起期待姐姐拆开礼物的那一刻吧！🎁🎁🎁

最后，我想用一句话来总结我对姐姐的祝福：愿你的每一天都充满爱和快乐，生日快乐！🎉🎂🎉

#生日快乐 #姐姐 #小红书 #分享 #祝福
